# CARF - Complex-Adaptive Reasoning Fabric
## Phase 2 Test Notebook

This notebook tests the CARF cognitive pipeline with DeepSeek LLM.

In [ ]:
# Setup - Add project root to path
import sys
sys.path.insert(0, '..')

# Load environment
from dotenv import load_dotenv
load_dotenv('../.env')

import asyncio
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)

print("CARF Environment Loaded")

## 1. Test LLM Configuration (DeepSeek)

In [ ]:
from src.core.llm import get_llm_config, get_chat_model

# Check configuration
config = get_llm_config()
print(f"Provider: {config.provider.value}")
print(f"Model: {config.model}")
print(f"Base URL: {config.base_url}")
print(f"API Key set: {'Yes' if config.api_key else 'No'}")

In [ ]:
# Test basic LLM call
from langchain_core.messages import HumanMessage

llm = get_chat_model(temperature=0.1, purpose="test")

response = await llm.ainvoke([HumanMessage(content="Say 'CARF is operational' if you can read this.")])
print(f"LLM Response: {response.content}")

## 2. Test Cynefin Router

In [ ]:
from src.workflows.router import CynefinRouter
from src.core.state import EpistemicState

router = CynefinRouter()

# Test Clear domain
state = EpistemicState(user_input="What is 2 + 2?")
result = await router.classify(state)
print(f"Query: '{state.user_input}'")
print(f"Domain: {result.cynefin_domain.value}")
print(f"Confidence: {result.domain_confidence:.2f}")
print(f"Hypothesis: {result.current_hypothesis}")
print()

In [ ]:
# Test Complicated domain
state = EpistemicState(user_input="Why did our server costs increase by 15% last quarter?")
result = await router.classify(state)
print(f"Query: '{state.user_input}'")
print(f"Domain: {result.cynefin_domain.value}")
print(f"Confidence: {result.domain_confidence:.2f}")
print(f"Hypothesis: {result.current_hypothesis}")

In [ ]:
# Test Complex domain
state = EpistemicState(user_input="How will users react to our new pricing strategy?")
result = await router.classify(state)
print(f"Query: '{state.user_input}'")
print(f"Domain: {result.cynefin_domain.value}")
print(f"Confidence: {result.domain_confidence:.2f}")
print(f"Hypothesis: {result.current_hypothesis}")

## 3. Test Causal Inference Engine

In [ ]:
from src.services.causal import CausalInferenceEngine

causal_engine = CausalInferenceEngine()

# Discover causal structure
query = "Our customer churn rate increased after we raised prices. What's the root cause?"
hypothesis, graph = await causal_engine.discover_causal_structure(query)

print("=== Causal Discovery ===")
print(f"Treatment: {hypothesis.treatment}")
print(f"Outcome: {hypothesis.outcome}")
print(f"Mechanism: {hypothesis.mechanism}")
print(f"Confounders: {hypothesis.confounders}")
print(f"Confidence: {hypothesis.confidence:.2f}")

In [ ]:
# Full causal analysis
result, graph = await causal_engine.analyze(
    "Marketing spend increased but sales didn't grow proportionally. Why?"
)

print("=== Causal Analysis ===")
print(f"Effect Estimate: {result.effect_estimate:.2f}")
print(f"Confidence Interval: {result.confidence_interval}")
print(f"Refutation Passed: {result.passed_refutation}")
print(f"\nInterpretation: {result.interpretation}")

## 4. Test Bayesian Active Inference

In [ ]:
from src.services.bayesian import ActiveInferenceEngine

bayesian_engine = ActiveInferenceEngine()

# Explore uncertain situation
result = await bayesian_engine.explore(
    "Should we expand into the European market? There's high uncertainty about regulatory environment and customer preferences."
)

print("=== Bayesian Exploration ===")
print(f"Initial Uncertainty: {result.uncertainty_before:.0%}")
print(f"Updated Uncertainty: {result.uncertainty_after:.0%}")
print(f"\nPrimary Hypothesis: {result.updated_belief.hypothesis}")
print(f"Confidence: {result.updated_belief.posterior:.0%}")
print(f"\nDesigned {len(result.probes_designed)} probes:")
for probe in result.probes_designed:
    print(f"  - {probe.description} (Info Gain: {probe.expected_information_gain:.0%}, Risk: {probe.risk_level})")
print(f"\nRecommended: {result.recommended_probe.description if result.recommended_probe else 'N/A'}")

## 5. Test Full Pipeline

In [ ]:
from src.workflows.graph import run_carf

# Test the full CARF pipeline
queries = [
    "What is the current stock price of AAPL?",  # Clear
    "Why did our conversion rate drop after the website redesign?",  # Complicated
    "What will be the impact of AI on our industry in 5 years?",  # Complex
]

for query in queries:
    print(f"\n{'='*60}")
    print(f"Query: {query}")
    print('='*60)
    
    result = await run_carf(query)
    
    print(f"Domain: {result.cynefin_domain.value}")
    print(f"Confidence: {result.domain_confidence:.2f}")
    print(f"Guardian Verdict: {result.guardian_verdict.value if result.guardian_verdict else 'N/A'}")
    print(f"\nReasoning Chain:")
    for step in result.reasoning_chain:
        print(f"  [{step.node_name}] {step.action}")
    print(f"\nResponse Preview: {result.final_response[:200] if result.final_response else 'N/A'}...")

## 6. Test API Server (Optional)

Run in terminal: `python -m src.main`

Then test:

In [ ]:
# Uncomment to test API
# import httpx
# 
# async with httpx.AsyncClient() as client:
#     response = await client.post(
#         "http://localhost:8000/query",
#         json={"query": "Why did our sales increase last month?"}
#     )
#     print(response.json())

## Summary

CARF Phase 2 Components:
- **DeepSeek LLM**: Cost-efficient reasoning
- **Cynefin Router**: Domain classification
- **Causal Engine**: Root cause analysis
- **Bayesian Engine**: Uncertainty exploration
- **Guardian**: Policy enforcement
- **HumanLayer**: HITL integration